# **ALL MODELS - POSTa GAN**

## **Import Dependencies**

In [1]:
# !rm -r /kaggle/working

In [1]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Number of devices: 2


In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tensorflow.keras import layers as L
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [3]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# **Data Loader**

In [4]:
# DataLoader
train_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/adversarial-tsrd',
    batch_size=128,
    image_size=(81, 81),
    label_mode="categorical",
    labels = "inferred",
    seed=42,
    validation_split=0.2,
    subset='training',
)

val_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/adversarial-tsrd',
    batch_size=128,
    image_size=(81, 81),
    label_mode="categorical",
    labels = "inferred",
    seed=42,
    validation_split=0.2,
    subset='validation',
)

Found 203364 files belonging to 58 classes.
Using 162692 files for training.
Found 203364 files belonging to 58 classes.
Using 40672 files for validation.


In [5]:
train_data.as_numpy_iterator().__next__()[0].shape

(128, 81, 81, 3)

In [6]:
train_data.as_numpy_iterator().__next__()[1].shape

(128, 58)

# **Model Prep**

In [7]:
NUM_CLASSES = len(os.listdir('/kaggle/input/adversarial-tsrd'))

In [8]:
# Create custom F1 Score Metrics

from keras import backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

In [9]:
steps_per_epoch = len(train_data)
EPOCHS = 128

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=4, min_lr=0.000001)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=8,# if val loss decreases for 8 epochs in a row, stop training
                                                  restore_best_weights = True)

# **Model Creations**

In [11]:
# Model

def create_model_1(shape=(81, 81, 3)):
    base_model = EfficientNetV2B0(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="EfficientNetV2B0")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.RMSprop(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [12]:
# Model

def create_model_2(shape=(81, 81, 3)):
    base_model = EfficientNetB0(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="EfficientNetB0")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.RMSprop(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [13]:
# Model

def create_model_3(shape=(81, 81, 3)):
    base_model = InceptionV3(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="InceptionV3")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.RMSprop(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [15]:
# Model

def create_model_4(shape=(81, 81, 3)):
    base_model = ResNet50(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="ResNet50")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.RMSprop(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [10]:
# Model

def create_model_5(shape=(81, 81, 3)):
    base_model = Xception(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
    for layer in base_model.layers[-3:]:
#         print(layer)
        layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(512, activation="relu", name="Dense-1")(x)
    x = L.Dropout(0.4, name="Final-Dropout")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="Xception")
    model.compile(
        loss="categorical_crossentropy",
        optimizer="Adam",
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

In [11]:
# Model

def create_model_6(shape=(81, 81, 3)):
    base_model = MobileNetV2(
        include_top = False,
        input_shape = shape,
        weights = "imagenet",
    )
    
    base_model.trainable = False
#     for layer in base_model.layers[-3:]:
# #         print(layer)
#         layer.trainable = True
        
    x = base_model.output
    x = L.Flatten(name="flatten")(x)
    x = L.Dense(NUM_CLASSES, activation="softmax", name="Output-layer")(x)
    
    model = tf.keras.models.Model(base_model.input, x, name="MobileNetV2")
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.experimental.RMSprop(),
        metrics = ["accuracy",tf.keras.metrics.AUC(name="auc"),
                               tf.keras.metrics.Recall(name="recall"),
                               tf.keras.metrics.Precision(name="precision"),
                              f1_score]
    )
    
    return model

# **Train Models**

In [17]:
model_1 = create_model_1()

# Fit the model
history_1 = model_1.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_1.save("/kaggle/working/EfficientNetV2B0.h5")
pd.DataFrame(history_1.history).to_csv("/kaggle/working/History-EfficientNetV2B0.csv")

24274472/24274472 [==============================] - 0s 0us/step
Epoch 1/128


2024-01-26 15:48:54.632476: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inEfficientNetV2B0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1272/1272 [==============================] - 369s 282ms/step - loss: 0.1898 - accuracy: 0.9467 - auc: 0.9972 - recall: 0.9330 - precision: 0.9691 - f1_score: 0.9478 - val_loss: 0.0487 - val_accuracy: 0.9850 - val_auc: 0.9993 - val_recall: 0.9828 - val_precision: 0.9881 - val_f1_score: 0.9855 - lr: 0.0010
Epoch 2/128
1272/1272 [==============================] - 60s 47ms/step - loss: 0.0433 - accuracy: 0.9866 - auc: 0.9995 - recall: 0.9848 - precision: 0.9890 - f1_score: 0.9869 - val_loss: 0.0464 - val_accuracy: 0.9866 - val_auc: 0.9990 - val_recall: 0.9858 - val_precision: 0.9883 - val_f1_score: 0.9870 - lr: 0.0010
Epoch 3/128
1272/1272 [==============================] - 61s 48ms/step - loss: 0.0258 - accuracy: 0.9919 - auc: 0.9997 - recall: 0.9910 - precision: 0.9932 - f1_score: 0.9921 - val_loss: 0.0290 - val_accuracy: 0.9915 - val_auc: 0.9994 - val_recall: 0.9908 - val_precision: 0.9928 - val_f1_score: 0.9918 - lr: 0.0010
Epoch 4/128
1272/1272 [==============================] - 60s 4

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
model_2 = create_model_2()

# Fit the model
history_2 = model_2.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_2.save("/kaggle/working/EfficientNetB0.h5")
pd.DataFrame(history_2.history).to_csv("/kaggle/working/History-EfficientNetB0.csv")

16705208/16705208 [==============================] - 0s 0us/step
Epoch 1/128


2024-01-26 18:02:01.785423: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inEfficientNetB0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1272/1272 [==============================] - 79s 56ms/step - loss: 0.2925 - accuracy: 0.9167 - auc: 0.9949 - recall: 0.8994 - precision: 0.9470 - f1_score: 0.9193 - val_loss: 0.1548 - val_accuracy: 0.9553 - val_auc: 0.9968 - val_recall: 0.9512 - val_precision: 0.9620 - val_f1_score: 0.9565 - lr: 0.0010
Epoch 2/128
1272/1272 [==============================] - 70s 55ms/step - loss: 0.0937 - accuracy: 0.9720 - auc: 0.9985 - recall: 0.9690 - precision: 0.9772 - f1_score: 0.9731 - val_loss: 0.0821 - val_accuracy: 0.9780 - val_auc: 0.9981 - val_recall: 0.9761 - val_precision: 0.9817 - val_f1_score: 0.9789 - lr: 0.0010
Epoch 3/128
1272/1272 [==============================] - 68s 53ms/step - loss: 0.0602 - accuracy: 0.9816 - auc: 0.9991 - recall: 0.9797 - precision: 0.9845 - f1_score: 0.9821 - val_loss: 0.0911 - val_accuracy: 0.9755 - val_auc: 0.9978 - val_recall: 0.9739 - val_precision: 0.9783 - val_f1_score: 0.9761 - lr: 0.0010
Epoch 4/128
1272/1272 [==============================] - 70s 54m

In [19]:
model_3 = create_model_3()

# Fit the model
history_3 = model_3.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_3.save("/kaggle/working/InceptionV3.h5")
pd.DataFrame(history_3.history).to_csv("/kaggle/working/History-InceptionV3.csv")

87910968/87910968 [==============================] - 1s 0us/step
Epoch 1/128
1272/1272 [==============================] - 83s 61ms/step - loss: 33.2795 - accuracy: 0.3399 - auc: 0.6831 - recall: 0.3393 - precision: 0.3412 - f1_score: 0.3405 - val_loss: 25.2633 - val_accuracy: 0.4678 - val_auc: 0.7503 - val_recall: 0.4671 - val_precision: 0.4697 - val_f1_score: 0.4684 - lr: 0.0010
Epoch 2/128
1272/1272 [==============================] - 76s 60ms/step - loss: 22.0891 - accuracy: 0.4795 - auc: 0.7559 - recall: 0.4790 - precision: 0.4808 - f1_score: 0.4803 - val_loss: 20.1344 - val_accuracy: 0.5193 - val_auc: 0.7747 - val_recall: 0.5188 - val_precision: 0.5208 - val_f1_score: 0.5198 - lr: 0.0010
Epoch 3/128
1272/1272 [==============================] - 57s 45ms/step - loss: 19.4258 - accuracy: 0.5301 - auc: 0.7806 - recall: 0.5297 - precision: 0.5314 - f1_score: 0.5305 - val_loss: 32.0627 - val_accuracy: 0.4612 - val_auc: 0.7414 - val_recall: 0.4609 - val_precision: 0.4619 - val_f1_score: 0

In [20]:
model_4 = create_model_4()

# Fit the model
history_4 = model_4.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_4.save("/kaggle/working/ResNet50.h5")
pd.DataFrame(history_4.history).to_csv("/kaggle/working/History-ResNet50.csv")

94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/128
1272/1272 [==============================] - 118s 88ms/step - loss: 0.7016 - accuracy: 0.8905 - auc: 0.9766 - recall: 0.8863 - precision: 0.9023 - f1_score: 0.8928 - val_loss: 0.2980 - val_accuracy: 0.9559 - val_auc: 0.9902 - val_recall: 0.9552 - val_precision: 0.9579 - val_f1_score: 0.9565 - lr: 0.0010
Epoch 2/128
1272/1272 [==============================] - 109s 86ms/step - loss: 0.1724 - accuracy: 0.9701 - auc: 0.9943 - recall: 0.9694 - precision: 0.9714 - f1_score: 0.9704 - val_loss: 0.2115 - val_accuracy: 0.9674 - val_auc: 0.9931 - val_recall: 0.9669 - val_precision: 0.9690 - val_f1_score: 0.9680 - lr: 0.0010
Epoch 3/128
1272/1272 [==============================] - 109s 86ms/step - loss: 0.0963 - accuracy: 0.9824 - auc: 0.9968 - recall: 0.9821 - precision: 0.9831 - f1_score: 0.9826 - val_loss: 0.1682 - val_accuracy: 0.9760 - val_auc: 0.9945 - val_recall: 0.9757 - val_precision: 0.9766 - val_f1_score: 0.97

In [12]:
model_5 = create_model_5()

# Fit the model
history_5 = model_5.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_5.save("/kaggle/working/Xception.h5")
pd.DataFrame(history_5.history).to_csv("/kaggle/working/History-Xception.csv")

83683744/83683744 [==============================] - 0s 0us/step
Epoch 1/128
1272/1272 [==============================] - 161s 120ms/step - loss: 1.0178 - accuracy: 0.7275 - auc: 0.9781 - recall: 0.6465 - precision: 0.8933 - f1_score: 0.7317 - val_loss: 0.5735 - val_accuracy: 0.8361 - val_auc: 0.9896 - val_recall: 0.7990 - val_precision: 0.9113 - val_f1_score: 0.8513 - lr: 0.0010
Epoch 2/128
1272/1272 [==============================] - 128s 100ms/step - loss: 0.4377 - accuracy: 0.8747 - auc: 0.9937 - recall: 0.8414 - precision: 0.9385 - f1_score: 0.8866 - val_loss: 0.4238 - val_accuracy: 0.8821 - val_auc: 0.9916 - val_recall: 0.8592 - val_precision: 0.9298 - val_f1_score: 0.8930 - lr: 0.0010
Epoch 3/128
1272/1272 [==============================] - 127s 100ms/step - loss: 0.3261 - accuracy: 0.9053 - auc: 0.9958 - recall: 0.8809 - precision: 0.9511 - f1_score: 0.9145 - val_loss: 0.3159 - val_accuracy: 0.9111 - val_auc: 0.9949 - val_recall: 0.8908 - val_precision: 0.9520 - val_f1_score: 0

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
model_6 = create_model_6()

# Fit the model
history_6 = model_6.fit(
    train_data,
    epochs = EPOCHS,
    steps_per_epoch = steps_per_epoch,
    validation_data = val_data,
    validation_steps = len(val_data),
    callbacks = [reduce_lr],
)

model_6.save("/kaggle/working/MobileNetV2.h5")
pd.DataFrame(history_6.history).to_csv("/kaggle/working/History-MobileNetV2.csv")

9406464/9406464 [==============================] - 0s 0us/step
Epoch 1/128
1272/1272 [==============================] - 66s 49ms/step - loss: 5.5571 - accuracy: 0.5136 - auc: 0.8278 - recall: 0.5065 - precision: 0.5313 - f1_score: 0.5168 - val_loss: 6.3847 - val_accuracy: 0.4995 - val_auc: 0.8141 - val_recall: 0.4960 - val_precision: 0.5046 - val_f1_score: 0.5003 - lr: 0.0010
Epoch 2/128
1272/1272 [==============================] - 66s 52ms/step - loss: 2.6741 - accuracy: 0.7103 - auc: 0.9084 - recall: 0.7068 - precision: 0.7184 - f1_score: 0.7127 - val_loss: 2.6569 - val_accuracy: 0.7225 - val_auc: 0.9113 - val_recall: 0.7195 - val_precision: 0.7300 - val_f1_score: 0.7247 - lr: 0.0010
Epoch 3/128
1272/1272 [==============================] - 66s 52ms/step - loss: 2.0828 - accuracy: 0.7710 - auc: 0.9290 - recall: 0.7686 - precision: 0.7768 - f1_score: 0.7728 - val_loss: 1.9398 - val_accuracy: 0.7929 - val_auc: 0.9366 - val_recall: 0.7903 - val_precision: 0.7972 - val_f1_score: 0.7937 - 

# **Results**

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.saving import load_model

class_names = os.listdir('/kaggle/input/adversarial-tsrd')

# **EfficientNetV2B0**

In [ ]:
metrics = list(history_1.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_1.history[metric])
        plt.plot(history_1.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **EfficientNetB0**

In [ ]:
metrics = list(history_2.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_2.history[metric])
        plt.plot(history_2.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **InceptionV3**

In [ ]:
metrics = list(history_3.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_3.history[metric])
        plt.plot(history_3.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **ResNet50**

In [ ]:
metrics = list(history_4.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_4.history[metric])
        plt.plot(history_4.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()

# **Xception**

In [ ]:
# metrics = list(history_5.history.keys())

# for metric in metrics:
#     if metric.startswith('val')==False and metric!='lr':
#         plt.figure(figsize=(6, 4))
#         plt.plot(history_5.history[metric])
#         plt.plot(history_5.history['val_'+metric])
#         plt.title('Model '+metric)
#         plt.ylabel(metric)
#         plt.xlabel('Epoch')
#         if metric.endswith('loss'):
#             plt.legend(['Train', 'Validation'], loc='upper right')
#         else:
#             plt.legend(['Train', 'Validation'], loc='lower right')

# plt.show()

# **MobileNetV2**

In [ ]:
metrics = list(history_6.history.keys())

for metric in metrics:
    if metric.startswith('val')==False and metric!='lr':
        plt.figure(figsize=(6, 4))
        plt.plot(history_6.history[metric])
        plt.plot(history_6.history['val_'+metric])
        plt.title('Model '+metric)
        plt.ylabel(metric)
        plt.xlabel('Epoch')
        if metric.endswith('loss'):
            plt.legend(['Train', 'Validation'], loc='upper right')
        else:
            plt.legend(['Train', 'Validation'], loc='lower right')

plt.show()